<a href="https://colab.research.google.com/github/vectara/example-notebooks/blob/main/notebooks/vectara-agentic-intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducing Vectara-Agentic: building AI assistants with Vectara

## About Vectara

[Vectara](https://vectara.com/) is the trusted GenAI RAG-as-a-service platform, providing a simple API for enterprise developers to created RAG applications that are enterprise ready.

See the [Vectara API documentation](https://docs.vectara.com/docs/) for more information on how to use the API.

The main benefits for using Vectara for a RAG application are:
* **Easy to use**: Vectara provides an end-to-end, fully functional, highly scalable and robust RAG pipeline, so as a user you don't have to code up these pieces and maintain them over time.
* **Scalable and Secure**: building GenAI applications may seem easy at first, but the DIY approach can become overwhelming beyond simple examples. Vectara provides instant scalablility to millions of documents, while maintaing data security and privacy, as well as latency SLAs.

## About Vectara-Agentic

Agentic RAG is a method that combines the strengths of RAG with autonomous agents.

With vanilla RAG, Vectara receives a user query, retrieves the most relevant facts from your data, and uses an LLM to generate the most accurate response based on those facts.

Agentic RAG leverages an LLM to “manage” the process of answering the user query via reasoning, planning, and a provided set of “tools”. Since a “manager” LLM-powered agent is in charge, it is smart enough to analyze the user query and properly call tools to obtain a comprehensive response to a complex user query.

[Vectara-agentic](https://vectara.github.io/vectara-agentic-docs/index.html) is a Python package that allows you to build Agentic RAG applications quickly with Vectara. 

## Getting Started

To get started you will need to:
1. [sign up](https://console.vectara.com/signup?utm_source=vectara&utm_medium=signup&utm_term=DevRel&utm_content=example-notebooks&utm_campaign=vectara-signup-DevRel-example-notebooks) (if you haven't already) and follow our [quickstart](https://docs.vectara.com/docs/quickstart) guide to create a corpus and an API key. 

2. Install vectara-agentic using: `pip install vectara-agentic`

In [1]:
%pip install -e ~/dev/py-vectara-agentic -q


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Then, setup your Vectara customer ID, corpus ID and API key as environment variables:

In [2]:
vectara_api_key = "<YOUR_VECTARA_API_KEY>"
vectara_corpus_id = "<YOUR_VECTARA_CORPUS_ID>"
vectara_customer_id = "<YOUR_VECTARA_CUSTOMER_ID>"

vectara_api_key = "zwt_RbZfGXvufiZWYLVpB-EGeLCY0g0jGURglxHWcw"
vectara_corpus_id = "20"
vectara_customer_id = "1169579801"

## The dataset

Vectara is a RAG-as-a-service platform, and in most cases data may be uploaded to the index ahead of time (e.g. by using [Airbyte](https://docs.airbyte.com/integrations/destinations/vectara), [Datavolo](https://vectara.com/blog/building-genai-enterprise-apps-with-vectara-and-datavolo/) or directly via Vectara's [indexing API](https://docs.vectara.com/docs/api-reference/indexing-apis/indexing) or using tools like [vectara-ingest](https://github.com/vectara/vectara-ingest)).

For this notebook we have uploaded [customer complaint data](https://huggingface.co/datasets/hpe-ai/customer-complaints) from HuggingFace into a Vectara corpus, including the following filtering attributes on the corpus to fit the dataset: `Company`, `State` and `Product`.

## Createing our AI Assistant

`vectara-agentic` provide a very flexible interface to create AI assistants and AI agents by defining one or more tools, custom instructions and personalizing the agent to a specific area.

You can read more about this [here](https://vectara.github.io/vectara-agentic-docs/usage.html).

In this notebook we demonstrate the simplest way to create an AI assistant, utilizing a single RAG Vectara tool, using the `from_corpus()` function:

In [3]:
from vectara_agentic.agent import Agent
import pprint

pp = pprint.PrettyPrinter(indent=4)

assistant1 = Agent.from_corpus(
    vectara_customer_id=vectara_customer_id,
    vectara_corpus_id=vectara_corpus_id,
    vectara_api_key=vectara_api_key,
    data_description="customer complaints",
    assistant_specialty="customer service representative",
    vectara_summarizer="mockingbird-1.0-2024-07-16"
)

Initializing vectara-agentic package...


Note that we've used Vectara's Mockingbird LLM by specifying the `vectara_summarizer` argument.

Let's ask the agent a simple question:

In [4]:
res = assistant1.chat("top complaints about credit scores exist")
pp.pprint(res)

('The top complaints about credit scores include:\n'
 '\n'
 '1. Inaccurate information on credit reports, such as accounts that have been '
 'paid but still show as delinquent.\n'
 '2. Incorrect reporting of account balances and past due amounts, which can '
 'negatively impact credit scores.\n'
 '3. Failure to remove or correct inaccurate information, even when the '
 'consumer has provided proof of payment.\n'
 '4. Unjust and unfair credit scoring practices, like penalizing individuals '
 'who pay off their balances every month.\n'
 '5. Fraudulent credit reporting, including illegal tampering with credit '
 'scores.\n'
 '6. Discriminatory credit scoring practices, such as prejudice against '
 'certain individuals or groups.\n'
 '\n'
 'These issues emphasize the need for accurate and fair credit reporting and '
 'the importance of consumers monitoring their credit scores to correct any '
 'errors.')


Now let's create a differnet AI assistant, this time with adding two other arguments to the Vectara RAG Tool:
1. A company name against which the complaint was filed.
2. The state of the complaint
We also use `verbose=True` to get more details about which steps the AI assistant is taking:

In [5]:
assistant2 = Agent.from_corpus(
    vectara_customer_id=vectara_customer_id,
    vectara_corpus_id=vectara_corpus_id,
    vectara_api_key=vectara_api_key,
    data_description="customer complaints",
    assistant_specialty="customer service representative",
    vectara_summarizer="mockingbird-1.0-2024-07-16",
    verbose=True,
    vectara_filter_fields=[
        {'type': str, 'name': 'Company', 'description': 'the company related to this complaint'},
        {'type': str, 'name': 'State', 'description': 'the state of the complaint, in shorthand format (e.g. TX for Texas)'}
    ]
)

In [6]:
res = assistant2.chat("top complaints about credit scores related to Experian Information Solutions Inc.")
pp.pprint(res)

Added user message to memory: top complaints about credit scores related to Experian Information Solutions Inc.
=== Calling Function ===
Calling function: vectara_20 with args: {"query":"top complaints about credit scores","Company":"Experian Information Solutions Inc."}
Got output: {'response': 'Based on the provided sources, the top complaints about credit scores are:\n\n1. Inaccurate credit scores: Different sites provide different credit scores, making it difficult to determine an accurate score [1].\n2. Unverified and erroneous information: Credit bureaus and companies are not providing proof and documents to support the information on credit reports, leading to incorrect and unverified information [2, 3, 5, 7, 8, 10].\n3. Failure to report payments: Mortgage companies are not reporting payments on time, which can lower credit scores and prevent refinancing [4].\n4. Identity theft: Victims of identity theft are not being taken seriously by credit bureaus, and unauthorized items ar

Notice how the AI assistant extracts the company name from the user query, and uses that as an argument to the RAG tool.

`Calling function: vectara_20 with args: {"query":"top complaints about credit scores","Company":"Experian Information Solutions Inc."}`

Similarly let's try with state:

In [7]:
assistant2.chat("top complaints about credit scores exist from California")
pp.pprint(res)

Added user message to memory: top complaints about credit scores exist from California
=== Calling Function ===
Calling function: vectara_20 with args: {"query":"top complaints about credit scores","State":"CA"}
Got output: {'response': 'Based on the provided sources, the top complaints about credit scores are:\n\n1. **Illegal tampering with credit scores**: Several consumers have reported that their credit scores dropped significantly without any justifiable reason, suggesting illegal tampering with their credit scores. [2, 5]\n2. **Inaccurate reporting of accounts**: Many consumers have disputed negative information on their credit reports, citing inaccurate reporting of accounts, including incorrect dates, amounts, and payment histories. [3, 9]\n3. **Fraudulent activity**: Some consumers have reported fraudulent activity, such as duplicate credit cards, which has led to unauthorized charges and a negative impact on their credit scores. [1, 4, 7]\n4. **Mishandling of accounts and pay

Similarly here we have:

`Calling function: vectara_20 with args: {"query":"top complaints about credit scores","State":"CA"}`

So the assistant is automatically identifying california as CA and calling the tool appropriately.